<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Trabajo final de master</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Estado de la computación cuántica en el aprendizaje por refuerzo y cómo aplicarla en DQN y Reinforce con Línea Base

## Red neuronal clásica aplicada a Reinforce en entorno Acrobot



Se importan las librerías necesarias para ejecutar el Jupyter Notebook

In [1]:
# Importamos timeit para tomar mediciones de tiempo de ejecución
import timeit

# Importamos codecarbon para tomar medidas de huella de carbono
from codecarbon import EmissionsTracker

In [2]:
import sys
# Para importar nuestras librerías, que están en otro directorio
sys.path.append("../librerias")

# Librería con circuito cuántico definido
import QVC

# Modelos de aprendizaje por refuerzo
import Model

# Agentes
import Agent

In [3]:
# Se importa la librería gymnasium para utilizar los entornos CartPole y Acrobot
import gym as gym

In [ ]:
import torch    # necesito la librería torch para la red neuronal

Se crea el entorno, en este caso CartPole

In [4]:
env = gym.envs.make("Acrobot-v1")

/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/gym/envs/registration.py:592: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Se definen los hiperparámetros que se utilizarán en el modelo de aprendizaje por refuerzo

In [5]:
# Hiperparámetros del modelo de aprendizaje por refuerzo
lr = 0.001            #Velocidad de aprendizaje
MEMORY_SIZE = 10000  #Máxima capacidad del buffer
MAX_EPISODES = 2000   #Número máximo de episodios (el agente debe aprender antes de llegar a este valor)
EPSILON = 1           #Valor inicial de epsilon
EPSILON_DECAY = .99   #Decaimiento de epsilon
GAMMA = 0.99          #Valor gamma de la ecuación de Bellman
BATCH_SIZE = 16       #Conjunto a coger del buffer para la red neuronal

Se define el circuito cuántico que sustituye a la red neuronal

In [ ]:
n_inputs = env.observation_space.shape[0]
n_outputs = env.action_space.n

In [8]:
net = torch.nn.Sequential(
            torch.nn.Linear(n_inputs, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, n_outputs),
            torch.nn.Softmax(dim=-1))

Se crea el modelo Reinforce con línea base

In [10]:
reinforce = Model.Reinforce(env, net, learning_rate=lr)

Se crea el agente, se le pasa como parámetro los dos modelos inicializados que contienen un circuito cuántico

In [12]:
agent = Agent.ReinforceAgent(env, reinforce)

Se prepara el entorno para iniciar las mediciones de CO2

In [13]:
# Se define la carpeta de salida de los archivos de emisiones
output_dir = './emisiones/emisiones_Classic_Reinforce_Acrobot'

# Se crea una instancia de EmissionsTracker y se configura la carpeta de salida
tracker = EmissionsTracker(output_dir=output_dir, log_level = "critical")

# Se inicializa el seguimiento de las emisiones
tracker.start()

Iniciamos el cronómetro para medir el tiempo del proceso

In [14]:
tiempo_inicio = timeit.default_timer()

Se inicia el entrenamiento del agente

In [ ]:
agent.train(gamma=GAMMA, max_episodes=MAX_EPISODES,
              batch_size=BATCH_SIZE)

Rellenando el buffer de repetición...
Entrenando...
Episodio 403 Recompensa media 41.24 Epsilon 0.017593337264871736		

Se calcula el tiempo de ejecución, lo que ha tardado el agente en entrenar

In [ ]:
tiempo_ejecucion = round(timeit.default_timer() - tiempo_inicio, 0)
print("Tiempo ejecución entrenamiento: " + str(int(tiempo_ejecucion/3600))
      + " horas, " + str(int((tiempo_ejecucion % 3600)/60)) + " minutos y "
      + str(int((tiempo_ejecucion % 3600)%60)) + " segundos")


Se calculan las emisiones que han podido implicar el proceso

In [ ]:
emisiones = tracker.stop()
print("Emisiones de CO2 (Kg)" + str(emisiones))

Se procede a graficar los resultados

In [ ]:
agent.plot_rewards()
agent.plot_loss()